# Hands-on at the ARMI Terminal

This tutorial will walk you through some exploration with ARMI on the command
line with the goal of exposing you to some of the capabilities
and organization of information in the ARMI system.

## Initializing and Exploring the ARMI Model
First we need to get some inputs. We built some from scratch in the, and we pick those up and use them here as well:


You can load these inputs using armi's ``init`` function. This will build an **Operator**, a **Reactor**, and an **Interface Stack** full of various interfaces.

In [ ]:
# you can only configure an app once
import armi
if not armi.isConfigured():
    armi.configure(armi.apps.App())

In [ ]:
o=armi.init(fName="../../../armi/tests/tutorials/anl-afci-177.yaml");

In [ ]:
core = o.r.core
core.getAssemblies()[:25] # only print the first 25

You can drill down the hierarchy for a particular assembly:

In [ ]:
core = o.r[0]
print(core)
assem = core[1]
print(assem)
block = assem[5]
print(block)
print(f"Block's parent is: {block.parent}")
components = block.getChildren()
print(components)
material = components[0].material
print(material)

## Exploring the *state* of the reactor
State can be explored using a variety of framework methods, as well as looking at state *parameters*. Let's first try out some methods to find out how much U-235 is in the model and what the average uranium enrichment is:

In [ ]:
u235 = core.getMass('U235')
u238 = core.getMass('U238')
print(f"The core contains {u235} grams of U-235")
print(f"The average fissile enrichment is {u235/(u235+u238)}")

That's how much U-235 is in the 1/3 core. If we want the total mass (including all nuclides), we can just leave the argument out:

In [ ]:
core.getMass()/1.e6

In [ ]:
core.getMass?

Furthermore, you can get a list of available methods by pressing the tab key. Try `core.` followed by `[Tab]`. Try out some options!

Next, lets find out what the number density of U235 is in a particular fuel block. We'll use the *FLAGS* system to select a particular type of block (in this case, a fuel block):

In [ ]:
from armi.reactor.flags import Flags
b = core.getFirstBlock(Flags.FUEL)
print(f"U-235 ndens: {b.getNumberDensity('U235'):.4e} (atoms/bn-cm)")
print(f"Block name: {b.getName()}")
print(f"Block type: {b.getType()}")

You can find lots of other details about this block with:

In [ ]:
b.printContents(includeNuclides=False)

## Modifying the state of the reactor
Each object in the Reactor model has a bunch of *state parameters* contained in it's special `.p` attribute, called it's *Parameter Collection*). The state parameters are defined both by the ARMI framework and the collection of plugins. For instance, you can look at the core's keff parameters or each individual block's power and multi-group flux parameters like this:

In [ ]:
print(b.p.power)
print(core.p.keff)
print(b.p.mgFlux)

As you might expect, the values are zero because we have not performed any physics calculations yet. We could run a physics plugin at this point to add physics state, but for this tutorial, we'll just apply dummy data. Here's a fake physics kernel that just sets a power distribution based on spatial location of each block (e.g. a spherical distribution):

In [ ]:
import numpy as np
midplane = core[0].getHeight()/2.0
center = np.array([0,0,midplane])
peakPower = 1e6
mgFluxBase = np.arange(5)
def setFakePower(core):
    for a in core:
        for b in a:
            vol = b.getVolume()
            coords = b.spatialLocator.getGlobalCoordinates()
            r = np.linalg.norm(abs(coords-center))
            fuelFlag = 10 if b.isFuel() else 1.0
            b.p.power = peakPower / r**2 * fuelFlag
            b.p.pdens = b.p.power/vol
            b.p.mgFlux = mgFluxBase*b.p.pdens
setFakePower(core)

In [ ]:
print(b.p.power)
print(b.p.pdens)

In [ ]:
import matplotlib.pyplot as plt
a = b.parent
z = [b.spatialLocator.getGlobalCoordinates()[2] for b in a]
power = a.getChildParamValues('power')
plt.plot(z,power,'.-')
plt.title("Fake power distribution on reactor")

We can take a look at the spatial distribution as well:

In [ ]:
core.plotFaceMap("power")

## Modifying number densities
Analysts frequently want to modify number densities. For example, if you needed to compute a coolant density coefficient, you could simply reduce the amount of coolant in the core. 

In [ ]:
sodiumBefore = core.getMass('NA')
print(f"Before: {sodiumBefore/1e6:.2f} MT Sodium")
for b in core.getBlocks():      # loop through all blocks
    refDens = b.getNumberDensity('NA23')
    b.setNumberDensity('NA23',refDens*0.98) # reduce Na density by 2%
sodiumAfter = core.getMass('NA')
print(f"After:  {sodiumAfter/1e6:.2f} MT Sodium")

If you analyze the keff with a physics plugin before and after, the change in the `core.p.keff` param would determine your density coefficient of reactivity. 

## Saving state to disk
During analysis, it's often useful to save the reactor state to disk in a database. The ARMI database package handles this, and writes it out to an [HDF-formatted](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) file. This is typically done automatically at each point in time in a normal simulation, and can also be done manually, like this:

In [ ]:
dbi = o.getInterface("database")
dbi.initDB()
dbi.database.writeToDB(o.r)

## Fuel management
One plugin that comes with the framework is the Fuel Handler. It attaches the Fuel Handler interface, which we can grab now to move fuel around. In a typical ARMI run, the detailed fuel management choices are specified by the user-input custom shuffle logic file. In this particular example, we will simply swap the 10 highest-power fuel assemblies with the 10 lowest-power ones. 

In [ ]:
from armi.physics.fuelCycle import fuelHandlers
fh = fuelHandlers.fuelHandlerFactory(o)

In [ ]:
moved = []
for n in range(10):
    high = fh.findAssembly(param="power", compareTo=1.0e6, blockLevelMax=True, exclusions=moved)
    low = fh.findAssembly(param="power", compareTo=0.0, blockLevelMax=True, exclusions=moved)
    fh.swapAssemblies(high, low)
    moved.extend([high, low])

In [ ]:
core.plotFaceMap("power")

We can write this new state to DB as well, since we've shuffled the fuel

In [ ]:
o.r.p.timeNode +=1
dbi.database.writeToDB(o.r)
dbi.database.close()

## Loading from the database
Once you have a database, you can load an operator object which has access to the reactor and case settings file.

Warning: While the reactor is in the same state as it was when we wrote the database, the operator may not be. This is because ARMI uses the parameter system to load state, and only the reactor and its children have parameters. Furthermore, anything not stored in a paramter is not written to the database.

In [ ]:
from armi.bookkeeping import db
databaseLocation = "anl-afci-177.h5"
cycle, timeNode = 0, 1
o = db.loadOperator(databaseLocation, cycle, timeNode)
r = o.r
cs = o.cs

We can see that the time node is what we expect (node 1), and there is some fission product mass since we loaded from a cycle after a depletion step.

In [ ]:
print(r.p.timeNode)
print(o.r.getFissileMass())

That's just a brief exploration of the data model. Hopefully it helped orient you to the underlying ARMI structure.